# Imports globales y funciones de utilidad

In [ ]:
import time
import torch
import itertools
import torchvision
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data.dataloader import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def get_dataloaders(train_transf, test_transf, batch_size):

  train_dataset = CIFAR10("data", train=True, download=True, transform=train_transf)
  test_dataset = CIFAR10("data", train=False, download=True, transform=test_transf)

  train_size = int(0.8 * len(train_dataset))
  valid_size = len(train_dataset) - train_size
  train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
  valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)

  return train_loader, valid_loader, test_loader

# Image Augmentation

La primera parte de este laboratorio consiste en expandir un poco el modelo de LeNet para tener más parámetros y luego explorar algunas técnicas de Image Augmentation (https://pytorch.org/vision/stable/transforms.html).

El modelo a implementar es el siguiente:


![Image](https://i.ibb.co/WxGgbmL/Capture.png)


In [ ]:
class CustomCNN(nn.Module):
  def __init__(self, in_channels):
    # in_channels: int, cantidad de canales de la imagen original
    super(CustomCNN, self).__init__()
    # Su implementacion

  def forward(self, x):
    # Su implementacion

## Funciones genericas para entrenar nuestros modelos

Vamos a utilizar las mismas funciones que implementamos en los laboratorios anteriores para entrenar y testear nuestros modelos.

In [ ]:
def train_epoch(training_model, loader, criterion, optim):
    training_model.train()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    for images, labels in loader:
      all_labels.extend(labels.numpy())  

      optim.zero_grad()

      predictions = training_model(images.to(device))
      all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

      loss = criterion(predictions, labels.to(device))
      
      loss.backward()
      optim.step()

      epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def validation_epoch(val_model, loader, criterion):
    val_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
      for images, labels in loader:
        all_labels.extend(labels.numpy())  

        predictions = val_model(images.to(device))
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels.to(device))

        epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100
  

def train_model(model, train_loader, test_loader, criterion, optim, number_epochs):
  train_history = []
  test_history = []
  accuracy_history = []

  for epoch in range(number_epochs):
      start_time = time.time()

      train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, train_acc, time.time() - start_time))

      start_time = time.time()
      test_loss, acc = validation_epoch(model, test_loader, criterion)
      test_history.append(test_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, test_loss, acc, time.time() - start_time))

## Entrenando modelos 

Comenzamos definiendo una seccion de código con valores por defecto de hiperparámetros que vamos a utilizar y luego entrenamos un modelo de la CNN definida anteriormente sin usar augmentation en los datos y otro haciendo uso del mismo.

https://pytorch.org/vision/stable/transforms.html

In [ ]:
# Global models config

BATCH_SIZE = 128
LR = 0.001
NUMBER_EPOCHS = 15
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
# Fijamos las semillas siempre para poder comparar.

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los dataloaders
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.ToTensor()
])

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

# Definimos el modelo y el optimizador
modelo_sin_aug = CustomCNN(3).to(device)
optimizer = torch.optim.Adam(modelo_sin_aug.parameters(), lr=LR)

# Entrenamos
train_model(modelo_sin_aug, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los datasets
test_transform = transforms.Compose([
    transforms.ToTensor()
])

# Definir transormaciones que vamos a aplicar al set de entrenamiento
train_transform = transforms.Compose([
    # ?? transforms.??? 
    transforms.ToTensor()
])
train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)


# Crear el modelo, optimizador y entrenarlo

## Evaluando los modelos en los datos de test

Vamos a comenzar evaluando en los datos de test normales y luego vamos a aplicar distintas transformaciones (para simular entornos más reales de datos) y vamos a ver la performance y robustez de los modelos que entrenamos anteriormente.

Primero agregar horizontal flip y luego vertical flip, qué pasa con los modelos?

In [ ]:
test_transform = transforms.Compose([
  transforms.ToTensor()                              
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

# Testear usando las funciones definidas anteriormente

In [ ]:
test_transform = transforms.Compose([
  # Flip Horizontal y volver a testear
  transforms.ToTensor()                              
])


In [ ]:
test_transform = transforms.Compose([
  # Flip Vertical y volver a testear
  transforms.ToTensor()                              
])


In [ ]:
# Otros tests que quieran probar...

# DenseNet


![Image](https://miro.medium.com/max/5164/1*_Y7-f9GpV7F93siM1js0cg.jpeg)

Link al paper original: [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

Algunas consideraciones del paper a tener en cuenta:

1. Batch normalization en los inputs de los bloques densos y las capas de transición.
2. ReLU en todos lados como funcion de activación.
3. El MLP al final de la red cuenta con una capa oculta de 512 neuronas
4. Las activaciones luego del tercer bloque denso tienen tamaño 4*4 (ejercicio, calcular a mano!)


Implementamos DenseNet para resolver el problema de CIFAR10


In [ ]:
class DenseBlock(nn.Module):
  def __init__(self, in_channels):
    super(DenseBlock, self).__init__()
    # Su implementacion
    
  def forward(self, x):
    # Su implementacion

In [ ]:
class TransitionLayer(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(TransitionLayer, self).__init__()
    # Su implementacion

  def forward(self, x):
    # Su implementacion

In [ ]:
class DenseNet(nn.Module):
	def __init__(self, n_classes):
		super(DenseNet, self).__init__()
    # Su implementacion

	def forward(self, x):
    # Su implementacion

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Creamos los datasets
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

densenet = DenseNet(10).to(device)
optimizer = torch.optim.Adam(densenet.parameters(), lr=LR)

train_loader, valid_loader, test_loader = get_dataloaders(train_transform, test_transform, BATCH_SIZE)

train_model(densenet, train_loader, valid_loader, criterion, optimizer, NUMBER_EPOCHS)

In [ ]:
test_transform = transforms.Compose([
  transforms.ToTensor()                              
])

_, _, test_loader = get_dataloaders(None, test_transform, BATCH_SIZE)

test_loss, accuracy = validation_epoch(densenet, test_loader, criterion)
print(f"DenseNet Test set: {test_loss:.6f} Loss. Accuracy {accuracy:.2f}%")